In [1]:
import os
import pandas as pd

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
model_path = "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
file_path = "/data/zju-48b/xhl/eval/my_eval/data/math500/test.jsonl"
df = pd.read_json(file_path, lines=True)
problem_list = df["problem"][:4]

texts_fast = []
texts_slow = []

for prob in problem_list:
    texts_fast.append("<|User|>Return your final response within \\boxed{}."+prob+"\n<|Assistant|><think>\nTo")

for prob in problem_list:
    texts_slow.append("<|User|>Return your final response within \\boxed{}."+prob+"\n<|Assistant|><think>\nAlright")

In [2]:
from vllm import LLM, SamplingParams
llm = LLM(
    model=model_path,
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1,
    enable_chunked_prefill=False
)

INFO 11-04 00:43:36 [utils.py:253] non-default args: {'disable_log_stats': True, 'enforce_eager': True, 'enable_steer_vector': True, 'enable_chunked_prefill': False, 'model': '/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/'}
INFO 11-04 00:43:37 [model.py:657] Resolved architecture: Qwen2ForCausalLM
INFO 11-04 00:43:37 [model.py:1746] Using max model len 131072
INFO 11-04 00:43:39 [scheduler.py:211] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 11-04 00:43:39 [vllm.py:414] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=2720351) INFO 11-04 00:43:40 [core.py:94] Initializing a V1 LLM engine (v0.1.dev10888+g9d4fd0da4.d20251031) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=F

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=2720351) INFO 11-04 00:44:14 [default_loader.py:314] Loading weights took 30.81 seconds
(EngineCore_DP0 pid=2720351) INFO 11-04 00:44:14 [steer_vector_model_runner_mixin.py:36] Initialized SteerVector worker manager
(EngineCore_DP0 pid=2720351) INFO 11-04 00:44:14 [steer_vector_model_runner_mixin.py:50] Wrapping model with steer vector support
(EngineCore_DP0 pid=2720351) INFO 11-04 00:44:14 [hidden_states_model_runner_mixin.py:90] Wrapped 28 decoder layers for hidden states capture
(EngineCore_DP0 pid=2720351) INFO 11-04 00:44:15 [gpu_model_runner.py:2971] Model loading took 3.3466 GiB and 30.986448 seconds
(EngineCore_DP0 pid=2720351) INFO 11-04 00:44:16 [gpu_worker.py:343] Available KV cache memory: 37.90 GiB
(EngineCore_DP0 pid=2720351) INFO 11-04 00:44:17 [kv_cache_utils.py:1247] GPU KV cache size: 1,419,360 tokens
(EngineCore_DP0 pid=2720351) INFO 11-04 00:44:17 [kv_cache_utils.py:1252] Maximum concurrency for 131,072 tokens per request: 10.83x
(EngineCore_DP0

In [3]:
answers_fast = llm.generate(
    texts_fast,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ),
)
answers_fast = [answer.outputs[0].text for answer in answers_fast]
qa_pairs_fast = [texts_fast[i] + answers_fast[i] for i in range(len(texts_fast))]

answers_slow = llm.generate(
    texts_slow,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ),
)
answers_slow = [answer.outputs[0].text for answer in answers_slow]
qa_pairs_slow = [texts_slow[i] + answers_slow[i] for i in range(len(texts_slow))]

Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

In [6]:
import gc

del llm
gc.collect()

544

In [7]:
import easysteer.hidden_states as hs
llm = LLM(
    model=model_path,
    task="embed", 
    tensor_parallel_size=1,
    enforce_eager=True,
    enable_prefix_caching=False,
    enable_chunked_prefill=False
)
all_hidden_states, _= hs.get_all_hidden_states(llm, qa_pairs_fast + qa_pairs_slow)

INFO 11-04 00:59:44 [utils.py:253] non-default args: {'task': 'embed', 'enable_prefix_caching': False, 'disable_log_stats': True, 'enforce_eager': True, 'enable_chunked_prefill': False, 'model': '/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/'}
INFO 11-04 00:59:44 [model.py:657] Resolved architecture: Qwen2ForCausalLM
INFO 11-04 00:59:44 [model.py:1746] Using max model len 131072
INFO 11-04 00:59:44 [vllm.py:414] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:45 [core.py:94] Initializing a V1 LLM engine (v0.1.dev10888+g9d4fd0da4.d20251031) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=a

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:47 [parallel_state.py:1325] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:47 [gpu_model_runner.py:2902] Starting to load model /data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/...
(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:48 [cuda.py:420] Using Flash Attention backend on V1 engine.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:49 [default_loader.py:314] Loading weights took 1.05 seconds
(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:49 [hidden_states_model_runner_mixin.py:90] Wrapped 28 decoder layers for hidden states capture
(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:50 [gpu_model_runner.py:2971] Model loading took 2.9110 GiB and 1.224297 seconds
(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:55 [gpu_worker.py:343] Available KV cache memory: 31.68 GiB
(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:55 [kv_cache_utils.py:1247] GPU KV cache size: 1,186,256 tokens
(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:55 [kv_cache_utils.py:1252] Maximum concurrency for 131,072 tokens per request: 9.05x
(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:55 [core.py:238] init engine (profile, create kv cache, warmup model) took 5.19 seconds
(EngineCore_DP0 pid=2737288) INFO 11-04 00:59:56 [vllm.py:414] Cudagraph is disabled under eager mode
INFO 11-04 00:59:56 [llm.py:346] Support

Adding requests:   0%|          | 0/8 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/8 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

In [9]:
from easysteer.steer import extract_pca_control_vector
control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states,
    positive_indices=list(range(4)), 
    negative_indices=list(range(4,8)),
    model_type="qwen2.5",
    method="center",
    token_pos=-1,
    normalize=False
)
control_vector.export_gguf("MATH500.gguf")

Computing PCA directions:   0%|          | 0/28 [00:00<?, ?it/s]

[2025-11-04 01:01:36] INFO pca.py:114: Layer 0: PCA on centered data explains 91.82945% of the variance
[2025-11-04 01:01:36] INFO pca.py:169: Layer 0: Direction corrected (flipped)
[2025-11-04 01:01:36] INFO pca.py:114: Layer 1: PCA on centered data explains 90.49839% of the variance
[2025-11-04 01:01:36] INFO pca.py:169: Layer 1: Direction corrected (flipped)
[2025-11-04 01:01:36] INFO pca.py:114: Layer 2: PCA on centered data explains 89.87948% of the variance
[2025-11-04 01:01:36] INFO pca.py:114: Layer 3: PCA on centered data explains 92.18152% of the variance
[2025-11-04 01:01:36] INFO pca.py:114: Layer 4: PCA on centered data explains 90.27144% of the variance
[2025-11-04 01:01:36] INFO pca.py:114: Layer 5: PCA on centered data explains 90.65833% of the variance
[2025-11-04 01:01:36] INFO pca.py:114: Layer 6: PCA on centered data explains 91.57656% of the variance
[2025-11-04 01:01:36] INFO pca.py:114: Layer 7: PCA on centered data explains 91.23605% of the variance
[2025-11-04 